In [1]:
import numpy as np
import pandas as pd
import pickle
import joblib
import os
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier

D:\Anaconda\lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
data = pd.read_excel('D:\Laxman\Projects\Fracture Analysis\Fracture Analysis Project\Dataset.xlsx')

In [3]:
data.head()

,SI.NO,DATE,NAME,AGE,SEX,ASSO MEDICAL PROB,H/O INJURY/SURGERY,DRUG HISTORY,FREQUENCY,avg
0,1,2017-02-01 00:00:00,PATIENT_1,29,Male,no,no,no,"115, 115, 113",114.30
1,2,2017-03-01 00:00:00,PATIENT_2,36,Female,no,no,no,"95,94,96",94.00
2,3,2017-03-01 00:00:00,PATIENT_3,37,Male,no,no,no,"80,82,81",81.00
3,4,2017-03-01 00:00:00,PATIENT_4,37,Female,no,no,no,"100, 101, 100",100.33
4,5,2017-03-01 00:00:00,PATIENT_5,38,female,no,no,no,99100100.4,99.80


In [4]:
data.shape

(1000, 10)

## Data understanding

In [5]:
data.isnull().sum()

SI.NO                 0
DATE                  0
NAME                  0
AGE                   0
SEX                   0
ASSO MEDICAL PROB     0
H/O INJURY/SURGERY    0
DRUG HISTORY          0
FREQUENCY             0
avg                   0
dtype: int64

In [6]:
data.columns

Index(['SI.NO', 'DATE', 'NAME', 'AGE', 'SEX', 'ASSO MEDICAL PROB',
       'H/O INJURY/SURGERY', 'DRUG HISTORY', 'FREQUENCY', 'avg'],
      dtype='object')

In [7]:
features_with_na = [feature for feature in data.columns if data[feature].isnull().sum()>1]
features_with_na

[]

In [8]:
data.describe()

,SI.NO,AGE,avg
count,1000.000000,1000.000000,1000.000000
mean,500.500000,56.520000,78.346700
std,288.819436,12.390445,12.431321
min,1.000000,29.000000,61.330000
25%,250.750000,45.000000,69.000000
50%,500.500000,56.500000,76.416500
75%,750.250000,67.000000,88.330000
max,1000.000000,85.000000,114.300000


## Basic Preprocessing and Cleaning

In [9]:
data['DATE']

0      2017-02-01 00:00:00
1      2017-03-01 00:00:00
2      2017-03-01 00:00:00
3      2017-03-01 00:00:00
4      2017-03-01 00:00:00
              ...         
995             12/29/2017
996             12/29/2017
997             12/31/2017
998             12/31/2017
999             12/31/2017
Name: DATE, Length: 1000, dtype: object

In [10]:
datetime.strptime('2017-02-01 00:00:00', '%Y-%m-%d %H:%M:%S').strftime('%d/%m/%Y')

'01/02/2017'

In [11]:
type(data['DATE'][0])

datetime.datetime

In [12]:
type(data['DATE'][999])

str

In [13]:
def try_parsing_date(text):
    if type(text) == str:
        return datetime.strptime(text, '%m/%d/%Y').strftime('%d/%m/%Y')
    else:
        return text.strftime('%d/%m/%Y')

In [14]:
data.DATE = [(try_parsing_date(i)) for i in data.DATE]

In [15]:
data['DATE']

0      01/02/2017
1      01/03/2017
2      01/03/2017
3      01/03/2017
4      01/03/2017
          ...    
995    29/12/2017
996    29/12/2017
997    31/12/2017
998    31/12/2017
999    31/12/2017
Name: DATE, Length: 1000, dtype: object

In [16]:
data['SEX'].value_counts()

SEX
Female    700
Male      260
female     20
male       20
Name: count, dtype: int64

In [17]:
data['SEX'].replace({'female':'Female', 'male':'Male'}, inplace=True)

C:\Users\prati\AppData\Local\Temp\ipykernel_8596\3629238160.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['SEX'].replace({'female':'Female', 'male':'Male'}, inplace=True)


In [18]:
data['SEX'].value_counts()

SEX
Female    720
Male      280
Name: count, dtype: int64

In [19]:
data['ASSO MEDICAL PROB'].value_counts()

ASSO MEDICAL PROB
no                                 480
No                                 160
yes(diabetes)                      100
yes(bp)                             60
yes(diabetes,bp)                    40
yes (diabetes)                      20
kidney stone                        20
yes(increase in heart rate)         20
Yes(Diabetes,bp)                    20
yes(bp dabetes)                     20
Yes(Diabetes,Blockage in Heart)     20
yes(diabetes,kidney stone)          20
yes (diabetes,heart blockage)       20
Name: count, dtype: int64

In [20]:
data['ASSO MEDICAL PROB'].replace({'No':'no', 
                                   'yes (diabetes)':'yes(diabetes)', 
                                   'Yes(Diabetes,bp)':'yes(diabetes,bp)',
                                  'Yes(Diabetes,Blockage in Heart)':'yes (diabetes,heart blockage)',
                                  'yes(bp dabetes)':'yes(diabetes,bp)'}, inplace=True)

C:\Users\prati\AppData\Local\Temp\ipykernel_8596\1771856808.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['ASSO MEDICAL PROB'].replace({'No':'no',


In [21]:
data['ASSO MEDICAL PROB'].value_counts()

ASSO MEDICAL PROB
no                               640
yes(diabetes)                    120
yes(diabetes,bp)                  80
yes(bp)                           60
yes (diabetes,heart blockage)     40
kidney stone                      20
yes(increase in heart rate)       20
yes(diabetes,kidney stone)        20
Name: count, dtype: int64

In [22]:
data['H/O INJURY/SURGERY'].value_counts()

H/O INJURY/SURGERY
no                         840
vericose vein surgery       20
uteres removal              20
kidney stone opreration     20
uterus surgery              20
yes(diverticulities)        20
shouler surgery             20
knee surgery                20
yes(open heart surgery)     20
Name: count, dtype: int64

In [23]:
data['H/O INJURY/SURGERY'].replace({'uteres surgery':'uterus surgery'}, inplace=True)

C:\Users\prati\AppData\Local\Temp\ipykernel_8596\1757368273.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['H/O INJURY/SURGERY'].replace({'uteres surgery':'uterus surgery'}, inplace=True)


In [24]:
data['H/O INJURY/SURGERY'].value_counts()

H/O INJURY/SURGERY
no                         840
vericose vein surgery       20
uteres removal              20
kidney stone opreration     20
uterus surgery              20
yes(diverticulities)        20
shouler surgery             20
knee surgery                20
yes(open heart surgery)     20
Name: count, dtype: int64

In [25]:
data['DRUG HISTORY'].value_counts()

DRUG HISTORY
no               760
yes              220
yes(ecosprin)     20
Name: count, dtype: int64

In [26]:
data.tail()

,SI.NO,DATE,NAME,AGE,SEX,ASSO MEDICAL PROB,H/O INJURY/SURGERY,DRUG HISTORY,FREQUENCY,avg
995,996,29/12/2017,PATIENT_996,72,Male,no,no,no,"68, 69, 68",68.30
996,997,29/12/2017,PATIENT_997,72,Male,"yes (diabetes,heart blockage)",shouler surgery,yes,"69,70,71",70.00
997,998,31/12/2017,PATIENT_998,74,Female,"yes(diabetes,bp)",knee surgery,yes,"63,62,62.6",62.53
998,999,31/12/2017,PATIENT_999,76,Male,"yes(diabetes,kidney stone)",no,no,"62,61,62.5",61.83
999,1000,31/12/2017,PATIENT_1000,85,Male,"yes (diabetes,heart blockage)",yes(open heart surgery),yes,"62,63,64",63.00


## Get Target Feature from existing feature

In [27]:
def get_target_feature(i):
    if (i<60):
        return 'Normal'
    elif (i>=60 and i<=100):
        return 'Osteopenia'
    else:
        return 'Osteoporotic'

In [28]:
data['output'] = data['avg'].astype(int).apply(get_target_feature)

In [29]:
data.head()

,SI.NO,DATE,NAME,AGE,SEX,ASSO MEDICAL PROB,H/O INJURY/SURGERY,DRUG HISTORY,FREQUENCY,avg,output
0,1,01/02/2017,PATIENT_1,29,Male,no,no,no,"115, 115, 113",114.30,Osteoporotic
1,2,01/03/2017,PATIENT_2,36,Female,no,no,no,"95,94,96",94.00,Osteopenia
2,3,01/03/2017,PATIENT_3,37,Male,no,no,no,"80,82,81",81.00,Osteopenia
3,4,01/03/2017,PATIENT_4,37,Female,no,no,no,"100, 101, 100",100.33,Osteopenia
4,5,01/03/2017,PATIENT_5,38,Female,no,no,no,99100100.4,99.80,Osteopenia


## Divide dataset in training and testing set

In [30]:
x = data.drop(['output', 'SI.NO', 'DATE', 'NAME', 'FREQUENCY', 'avg'], axis=1)
y = data['output']

In [31]:
x.head()

,AGE,SEX,ASSO MEDICAL PROB,H/O INJURY/SURGERY,DRUG HISTORY
0,29,Male,no,no,no
1,36,Female,no,no,no
2,37,Male,no,no,no
3,37,Female,no,no,no
4,38,Female,no,no,no


In [32]:
x.SEX = x.SEX.map( {'Male' : 1, 'Female' : 0 } )

x['ASSO MEDICAL PROB'] = x['ASSO MEDICAL PROB'].map( {'no' : 0, 
                                               'yes(diabetes)' : 1, 
                                               'yes(diabetes,bp)' : 2,
                                               'yes(bp)' : 3,
                                               'yes (diabetes,heart blockage)' : 4,
                                               'kidney stone' : 5,
                                               'yes(increase in heart rate)' : 6,
                                               'yes(diabetes,kidney stone)' : 7,
                                               } )

x['H/O INJURY/SURGERY'] = x['H/O INJURY/SURGERY'].map( {'no' : 0, 
                                               'vericose vein surgery' : 1, 
                                               'uteres removal' : 2,
                                               'kidney stone opreration' : 3,
                                               'uterus surgery' : 4,
                                               'yes(diverticulities)' : 5,
                                               'shouler surgery' : 6,
                                               'knee surgery' : 7,
                                               'yes(open heart surgery)' : 8,
                                               } )

x['DRUG HISTORY'] = x['DRUG HISTORY'].map( {'no' : 0, 
                                               'yes' : 1, 
                                               'yes(ecosprin)' : 2,
                                               } )


In [33]:
x.head()

,AGE,SEX,ASSO MEDICAL PROB,H/O INJURY/SURGERY,DRUG HISTORY
0,29,1,0,0,0
1,36,0,0,0,0
2,37,1,0,0,0
3,37,0,0,0,0
4,38,0,0,0,0


In [34]:
y.value_counts()

output
Osteopenia      980
Osteoporotic     20
Name: count, dtype: int64

## Model Fitting, Training and Prediction

In [35]:
model = DecisionTreeClassifier()
model.fit(x, y)

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [36]:
model.predict( [ [35,1,0,2,1] ] )

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array(['Osteopenia'], dtype=object)

In [37]:
# from sklearn.preprocessing import StandardScaler

# sc_x = StandardScaler()
# x_train = sc_x.fit_transform(x_train) 
# x_test = sc_x.transform(x_test)
  
# print (x_train[0:10, :])

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
model = DecisionTreeClassifier()
model.fit(x_train, y_train)

predictions = model.predict(x_test)

score = accuracy_score(y_test, predictions)
score

1.0

In [39]:
from sklearn.linear_model import LogisticRegression

model2 = LogisticRegression()
model2.fit(x_train, y_train)

predictions = model2.predict(x_test)

score = accuracy_score(y_test, predictions)
score

1.0

In [40]:
model

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


## Model Dumping and Loading

In [41]:
pickle.dump(model, open('Fracture_Detection.pkl','wb'))

In [42]:
load_model = pickle.load(open('Fracture_Detection.pkl','rb'))

In [43]:
prediction = load_model.predict([ [35,1,0,2,1] ])
prediction

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array(['Osteopenia'], dtype=object)

In [44]:
y_pred_loaded = load_model.predict(x_test)

In [45]:
score_loaded = accuracy_score(y_test, y_pred_loaded)
score_loaded

1.0